In [106]:
# coding:utf-8
# Author: yuxiang Zeng
from torch.utils.data import Dataset, DataLoader
from torch.nn import *
from tqdm import *
from time import time
import numpy as np
import pickle as pk
import pandas as pd
import torch as t

In [107]:
class Args:
    def __init__(self, density):
        self.density = density
        self.dimension = 5
        self.batch_size = 128
        self.epochs = 50
        self.devices = 'gpu'
        self.verbose = 0

In [108]:
def ErrMetrics(realVec, estiVec):
    realVec = t.tensor(realVec)
    estiVec = t.tensor(estiVec)

    absError = t.abs(estiVec - realVec)

    MAE = t.mean(absError)

    RMSE = t.linalg.norm(absError) / t.sqrt(t.tensor(absError.shape[0]))

    NMAE = MAE / (t.sum(realVec) / absError.shape[0])

    relativeError = absError / realVec

    MRE = t.tensor(np.percentile(relativeError, 50))  # Mean Relative Error
    NPRE = t.tensor(np.percentile(relativeError, 90))  #

    return MAE, RMSE, NMAE, MRE, NPRE

In [109]:
class Basic_SVD(Module):
    def __init__(self, dimension):
        super().__init__()
        self.user_embed = t.nn.Embedding(339, dimension)
        self.item_embed = t.nn.Embedding(5825, dimension)
        self.activation = t.nn.ReLU()
        self.pred_layer = t.nn.Linear(dimension, 1)

    def forward(self, userIdx, itemIdx):
        user_embeds = self.user_embed(userIdx)
        item_embeds = self.item_embed(itemIdx)

        # prediction = self.activation(self.pred_layer(user_embeds * item_embeds))
        prediction = self.pred_layer(user_embeds * item_embeds)

        prediction[prediction < 0] = 0
        return prediction.flatten(), user_embeds, item_embeds

In [110]:
def test_train_split(tensor, args):
    quantile = np.percentile(tensor, q=100)
    tensor[tensor > quantile] = 0
    tensor /= quantile
    density = args.density

    mask = np.random.rand(*tensor.shape).astype('float32')  # [0, 1]

    mask[mask > density] = 1
    mask[mask < density] = 0

    train_Tensor = tensor * (1 - mask)

    size = int(0.05 * np.prod(tensor.shape))

    trIdx, tcIdx = mask.nonzero()
    p = np.random.permutation(len(trIdx))
    trIdx, tcIdx = trIdx[p], tcIdx[p]

    vrIdx, vcIdx = trIdx[:size], tcIdx[:size]
    trIdx, tcIdx = trIdx[size:], tcIdx[size:]

    valid_Tensor = np.zeros_like(tensor)
    test_Tensor = np.zeros_like(tensor)

    valid_Tensor[vrIdx, vcIdx] = tensor[vrIdx, vcIdx]

    test_Tensor[trIdx, tcIdx] = tensor[trIdx, tcIdx]

    return train_Tensor, valid_Tensor, test_Tensor, quantile

In [111]:
class QoSdataset(Dataset):
    def __getitem__(self, index):
        output = self.idx[index]
        userIdx, itemIdx, value = t.as_tensor(output[0]).long(), t.as_tensor(output[1]).long(), output[2]
        return userIdx, itemIdx, value

    def __init__(self, data):
        super().__init__()
        self.data = np.array(data)
        self.idx = self.get_index(self.data)

    def __len__(self):
        return len(self.idx)

    @staticmethod
    def get_index(data):
        userIdx, itemIdx = data.nonzero()
        value = []
        for i in trange(len(userIdx)):
            value.append(data[userIdx[i], itemIdx[i]])
        index = np.transpose([userIdx, itemIdx, np.array(value)])
        return t.tensor(index)

In [112]:
def get_dataloader(train_set, valid_set, test_set):
    agg = False

    train_loader = DataLoader(
        train_set,
        batch_size=args.batch_size,
        drop_last=True,
        shuffle=False,
        pin_memory=True,
        num_workers=0 if not agg else 4,
        prefetch_factor=2 if not agg else 2
    )

    valid_loader = DataLoader(
        valid_set,
        batch_size=args.batch_size,
        drop_last=True,
        shuffle=False,
        pin_memory=True,
        num_workers=0 if not agg else 4,
        prefetch_factor=2 if not agg else 2
    )

    test_loader = DataLoader(
        test_set,
        batch_size=args.batch_size,
        drop_last=True,
        shuffle=False,  # 测试集没必要打乱
        pin_memory=True,
        num_workers=0 if not agg == 'Windows' else 8,
        prefetch_factor=2 if not agg == 'Windows' else 4
    )

    return train_loader, valid_loader, test_loader

In [113]:
from utility.utils import log


def train(model, train_loader, valid_loader):
    loss_function = t.nn.L1Loss()

    learning_rate = 1e-3
    optimizer_model = t.optim.Adam(model.parameters(), lr=learning_rate)

    if args.devices == 'gpu':
        model = model.cuda()
        loss_function = loss_function.cuda()

    validBestMAE, validBestRMSE, validBestNMAE, validBestMRE, validBestNPRE = 1e5, 1e5, 1e5, 1e5, 1e5

    params = None
    for epoch in range(args.epochs):

        t.set_grad_enabled(True)

        for train_Batch in train_loader:
            userIdx, itemIdx, mVal = train_Batch
            if args.devices == 'gpu':
                userIdx, itemIdx, mVal = userIdx.cuda(), itemIdx.cuda(), mVal.cuda()
            pred = model.forward(userIdx, itemIdx)
            loss = loss_function(pred, mVal)
            optimizer_model.zero_grad()
            loss.backward()
            optimizer_model.step()

        # 设定为评估状态
        t.set_grad_enabled(False)

        reals, preds = [], []
        for valid_Batch in valid_loader:
            userIdx, itemIdx, mVal = valid_Batch
            if args.devices == 'gpu':
                userIdx, itemIdx = userIdx.cuda(), itemIdx.cuda()
            pred = model.forward(userIdx, itemIdx)
            reals += mVal.tolist()
            preds += pred.tolist()

        reals = np.array(reals)
        preds = np.array(preds)

        validMAE, validRMSE, validNMAE, validMRE, validNPRE = ErrMetrics(reals * quantile, preds * quantile)

        if args.verbose and ((epoch + 1) % args.verbose == 0 or (epoch + 1) == args.epochs):
            log(f'Epoch {(epoch + 1):2d} : MAE : {validMAE:5.4f}  RMSE : {validRMSE:5.4f}  NMAE : {validNMAE:5.4f}  MRE : {validMRE:5.4f}  NPRE : {validNPRE:5.4f}')

        if validMAE < validBestMAE:
            params = model.state_dict()

    return params

In [114]:
def test(model, test_loader):
    reals, preds = [], []
    # 开始测试
    for testBatch in test_loader:
        userIdx, itemIdx, mVal = testBatch
        if args.devices == 'gpu':
            userIdx, itemIdx = userIdx.cuda(), itemIdx.cuda()
        pred = model.forward(userIdx, itemIdx)
        reals += mVal.tolist()
        preds += pred.tolist()

    reals = np.array(reals)
    preds = np.array(preds)
    testMAE, testRMSE, testNMAE, testMRE, testNPRE = ErrMetrics(reals * quantile , preds * quantile)

    log(f'Result : MAE : {testMAE:5.4f}  RMSE : {testRMSE:5.4f}  NMAE : {testNMAE:5.4f}  MRE : {testMRE:5.4f}  NPRE : {testNPRE:5.4f}')

*****

In [115]:
args = Args(0.1)
args.verbose = 10
args.epochs = 55

In [116]:
# df = np.array(pk.load(open('./datasets/data/WSDREAM/rt.pk', 'rb')))
# train_Tensor, valid_Tensor, test_Tensor, quantile = test_train_split(df, args)

In [117]:
# (train_Tensor != 0).sum(), (valid_Tensor != 0).sum(), (test_Tensor != 0).sum()

In [118]:
# train_set = QoSdataset(train_Tensor)
# valid_set = QoSdataset(valid_Tensor)
# test_set = QoSdataset(test_Tensor)

In [119]:
# net = Basic_SVD(args.dimension)
# train_loader, valid_loader, test_loader = get_dataloader(train_set, valid_set, test_set)
# params = train(net, train_loader, valid_loader)

In [120]:
# net.load_state_dict(params)
# test(net, test_loader)

In [121]:
# SISA2
def random_split_to_shard2(tensor, args):
    split_Tensor = []
    # print('456')

    train_Tensor = tensor

    idx = np.arange(339, dtype='int64')
    p = np.random.permutation(len(idx))
    idx = idx[p]

    size = int(339 * 0.1)
    idx = idx[:size]

    train_Tensor = np.zeros_like(tensor)
    train_Tensor[idx] = tensor[idx]
    return train_Tensor, idx

def learn(model, train_loader, valid_loader):
    loss_function = t.nn.L1Loss()
    # args.epochs = 1
    learning_rate = 1e-3
    optimizer_model = t.optim.Adam(model.parameters(), lr=learning_rate)
    model = model.cuda()
    loss_function = loss_function.cuda()
    for epoch in range(args.epochs):
        t.set_grad_enabled(True)
        for train_Batch in train_loader:
            userIdx, itemIdx, mVal = train_Batch
            userIdx, itemIdx, mVal = userIdx.cuda(), itemIdx.cuda(), mVal.cuda()
            pred, user_embedding, item_embedding = model.forward(userIdx, itemIdx)
            loss = loss_function(pred, mVal)
            optimizer_model.zero_grad()
            loss.backward()
            optimizer_model.step()

        # 设定为评估状态
        t.set_grad_enabled(False)

    return user_embedding, item_embedding, model.state_dict()

In [122]:
df = np.array(pk.load(open('./datasets/data/WSDREAM/rt.pk', 'rb')))
train_Tensor, valid_Tensor, test_Tensor, quantile = test_train_split(df, args)
train_Tensor, idx = random_split_to_shard2(df, args)
args.batch_size = 192225
train_set = QoSdataset(train_Tensor)
valid_set = QoSdataset(valid_Tensor)
test_set = QoSdataset(test_Tensor)
net = Basic_SVD(args.dimension)
train_loader, valid_loader, test_loader = get_dataloader(train_set, valid_set, test_set)
args.epochs = 1
user_embedding, item_embedding, _ = learn(net, train_loader, valid_loader)

100%|██████████| 1678360/1678360 [00:00<00:00, 3421495.29it/s]


In [123]:
train_set.idx.shape

torch.Size([192225, 3])

In [124]:
idx = sorted(idx)
for i in idx:
    print(i, end=' ')

1 5 8 13 31 45 51 54 62 77 90 95 103 104 107 112 157 158 166 167 200 209 211 222 228 239 244 272 277 280 296 318 327 

In [125]:
f = t.nn.Linear(args.dimension, 1).cuda()
y = f(user_embedding * item_embedding)
y * 19.9

tensor([[36.3289],
        [-3.2075],
        [14.4693],
        ...,
        [-3.9652],
        [-2.5721],
        [ 7.3932]], device='cuda:0')

In [126]:
user_embedding.cpu().numpy().shape, item_embedding.cpu().numpy().shape

((192225, 5), (192225, 5))

In [127]:
user_embedding.cpu().numpy()

array([[-1.8154032 , -0.8046755 ,  0.5940565 ,  0.52667314, -0.12243475],
       [-1.8154032 , -0.8046755 ,  0.5940565 ,  0.52667314, -0.12243475],
       [-1.8154032 , -0.8046755 ,  0.5940565 ,  0.52667314, -0.12243475],
       ...,
       [ 0.22662273, -2.4708698 ,  0.9294187 ,  0.4272715 , -0.46499878],
       [ 0.22662273, -2.4708698 ,  0.9294187 ,  0.4272715 , -0.46499878],
       [ 0.22662273, -2.4708698 ,  0.9294187 ,  0.4272715 , -0.46499878]],
      dtype=float32)

In [128]:
item_embedding.cpu().numpy()

array([[ 2.647503  ,  0.32394072, -0.39682087, -0.6036693 , -0.7839168 ],
       [-0.632833  , -1.1172096 ,  0.68209636, -0.02556576, -0.07542175],
       [ 0.67464656, -0.8494805 ,  1.091489  , -1.7129805 , -0.20339127],
       ...,
       [ 1.4186572 , -0.3099547 , -0.18475732,  0.54770195,  0.89754623],
       [ 0.6377097 ,  0.32385835, -0.23696978,  2.012862  ,  0.3786234 ],
       [-0.5017906 ,  0.07678311,  1.1613884 ,  0.14551407,  0.26140904]],
      dtype=float32)

In [129]:
user_embedding * item_embedding

tensor([[-4.8063, -0.2607, -0.2357, -0.3179,  0.0960],
        [ 1.1488,  0.8990,  0.4052, -0.0135,  0.0092],
        [-1.2248,  0.6836,  0.6484, -0.9022,  0.0249],
        ...,
        [ 0.3215,  0.7659, -0.1717,  0.2340, -0.4174],
        [ 0.1445, -0.8002, -0.2202,  0.8600, -0.1761],
        [-0.1137, -0.1897,  1.0794,  0.0622, -0.1216]], device='cuda:0')

In [130]:
temp = None
for name, parameter in net.named_parameters():
    print(name)
    print(parameter)
    print(parameter.shape)
    print('-' * 80)

user_embed.weight
Parameter containing:
tensor([[ 1.5765, -0.2418, -0.5091,  0.8081, -0.3201],
        [-1.8144, -0.8037,  0.5931,  0.5257, -0.1234],
        [ 0.6296, -0.8731, -1.1572,  0.0776,  0.7717],
        ...,
        [-0.2629, -0.8242,  0.2592, -0.8079,  1.5144],
        [ 1.4680, -1.2757, -0.2142, -2.7597,  0.3174],
        [-0.5477,  0.5541,  0.7850, -0.6887, -1.3640]], device='cuda:0',
       requires_grad=True)
torch.Size([339, 5])
--------------------------------------------------------------------------------
item_embed.weight
Parameter containing:
tensor([[ 2.6465,  0.3229, -0.3958, -0.6047, -0.7849],
        [-0.6318, -1.1162,  0.6811, -0.0246, -0.0745],
        [ 0.6736, -0.8501,  1.0905, -1.7120, -0.2025],
        ...,
        [ 1.4177, -0.3110, -0.1838,  0.5467,  0.8966],
        [ 0.6367,  0.3229, -0.2360,  2.0119,  0.3776],
        [-0.5008,  0.0758,  1.1604,  0.1465,  0.2604]], device='cuda:0',
       requires_grad=True)
torch.Size([5825, 5])
--------------------

In [131]:
net.state_dict()['user_embed.weight'].cpu().numpy()

array([[ 1.5765132 , -0.24181476, -0.5090597 ,  0.80809194, -0.32008168],
       [-1.8144032 , -0.80367553,  0.5930565 ,  0.52567315, -0.12343416],
       [ 0.6296369 , -0.87307435, -1.1572306 ,  0.07763115,  0.77166176],
       ...,
       [-0.262866  , -0.82421356,  0.25917768, -0.8078682 ,  1.5144199 ],
       [ 1.4680147 , -1.2757335 , -0.21419285, -2.7596633 ,  0.3173996 ],
       [-0.54773873,  0.5540975 ,  0.78499675, -0.68874353, -1.3639877 ]],
      dtype=float32)

In [132]:
# net.state_dict()['item_embed.weight'].cpu().numpy()

In [133]:
np.array(idx)

array([  1,   5,   8,  13,  31,  45,  51,  54,  62,  77,  90,  95, 103,
       104, 107, 112, 157, 158, 166, 167, 200, 209, 211, 222, 228, 239,
       244, 272, 277, 280, 296, 318, 327], dtype=int64)

In [134]:
trained = []
non_trained = []
arr = net.state_dict()['user_embed.weight'].cpu().numpy()
for i in range(len(arr)):
    if i in idx:
        trained.append(arr[i].tolist())
    else:
        non_trained.append(arr[i].tolist())

In [135]:
np.array(trained)

array([[-1.81440318, -0.80367553,  0.5930565 ,  0.52567315, -0.12343416],
       [-0.13140589, -1.42670798,  1.39358091, -0.11256768, -1.67763603],
       [ 1.75348532,  0.40418062,  1.26916695,  1.28193045,  0.90981156],
       [ 0.81482798,  0.59471709,  1.26239395,  1.3121177 , -0.82508183],
       [ 0.09462714,  0.20207307, -0.92027861,  0.39040729,  0.02768595],
       [-0.32375687, -1.83446753,  0.3009524 , -0.70259476,  0.67990357],
       [ 1.81397641, -2.15060186, -0.40559715, -0.0374691 , -0.68198574],
       [ 1.47702444,  0.21240695, -0.96304923,  0.54820305,  1.99177039],
       [ 0.3561098 , -1.07826507, -1.59564006, -1.29226232,  0.7676031 ],
       [ 0.67558384,  0.01700601, -0.89007878, -3.24522567,  0.09809425],
       [ 0.23596637, -0.63896775,  1.19222116, -0.03510421, -0.18140012],
       [ 0.9647581 , -0.18848954,  0.70268506, -0.90309221, -0.78558004],
       [ 1.12619054, -1.80314028, -1.33996403, -0.03029682, -0.3700164 ],
       [ 1.15867579, -0.30095524, -1.0

In [136]:
user_embed = []
net = net.cuda()
for i in range(339):
    user_embed.append(net.user_embed(t.tensor(i).cuda()).cpu().tolist())

In [137]:
for i in idx:
    print(i, end=' ')

1 5 8 13 31 45 51 54 62 77 90 95 103 104 107 112 157 158 166 167 200 209 211 222 228 239 244 272 277 280 296 318 327 

In [138]:
np.array(user_embed)

array([[ 1.57651317, -0.24181476, -0.50905973,  0.80809194, -0.32008168],
       [-1.81440318, -0.80367553,  0.5930565 ,  0.52567315, -0.12343416],
       [ 0.62963688, -0.87307435, -1.15723062,  0.07763115,  0.77166176],
       ...,
       [-0.26286599, -0.82421356,  0.25917768, -0.80786818,  1.51441991],
       [ 1.46801472, -1.27573347, -0.21419285, -2.75966334,  0.31739959],
       [-0.54773873,  0.55409747,  0.78499675, -0.68874353, -1.36398768]])

In [139]:
trained, non_trained = [], []
for i in range(len(user_embed)):
    if i in idx:
        trained.append(user_embed[i])
    else:
        non_trained.append(user_embed[i])

In [140]:
np.array(trained)

array([[-1.81440318, -0.80367553,  0.5930565 ,  0.52567315, -0.12343416],
       [-0.13140589, -1.42670798,  1.39358091, -0.11256768, -1.67763603],
       [ 1.75348532,  0.40418062,  1.26916695,  1.28193045,  0.90981156],
       [ 0.81482798,  0.59471709,  1.26239395,  1.3121177 , -0.82508183],
       [ 0.09462714,  0.20207307, -0.92027861,  0.39040729,  0.02768595],
       [-0.32375687, -1.83446753,  0.3009524 , -0.70259476,  0.67990357],
       [ 1.81397641, -2.15060186, -0.40559715, -0.0374691 , -0.68198574],
       [ 1.47702444,  0.21240695, -0.96304923,  0.54820305,  1.99177039],
       [ 0.3561098 , -1.07826507, -1.59564006, -1.29226232,  0.7676031 ],
       [ 0.67558384,  0.01700601, -0.89007878, -3.24522567,  0.09809425],
       [ 0.23596637, -0.63896775,  1.19222116, -0.03510421, -0.18140012],
       [ 0.9647581 , -0.18848954,  0.70268506, -0.90309221, -0.78558004],
       [ 1.12619054, -1.80314028, -1.33996403, -0.03029682, -0.3700164 ],
       [ 1.15867579, -0.30095524, -1.0

In [141]:
np.array(non_trained)

array([[ 1.57651317, -0.24181476, -0.50905973,  0.80809194, -0.32008168],
       [ 0.62963688, -0.87307435, -1.15723062,  0.07763115,  0.77166176],
       [ 0.96675229,  1.15178859,  0.12776124, -0.05298454, -1.41787803],
       ...,
       [-0.26286599, -0.82421356,  0.25917768, -0.80786818,  1.51441991],
       [ 1.46801472, -1.27573347, -0.21419285, -2.75966334,  0.31739959],
       [-0.54773873,  0.55409747,  0.78499675, -0.68874353, -1.36398768]])

In [142]:

temp = None
for name, parameter in net.named_parameters():
    # print(name)
    # print(parameter)
    # print(parameter.shape)
    print('-' * 80)

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------


In [150]:
args.epochs =  1
user_embedding, item_embedding, params = learn(net, train_loader, valid_loader)
net.load_state_dict(params)
user_embed = []
for i in range(339):
    user_embed.append(net.user_embed(t.tensor(i).cuda()).cpu().tolist())
trained, non_trained = [], []
for i in range(len(user_embed)):
    if i in idx:
        trained.append(user_embed[i])
    else:
        non_trained.append(user_embed[i])
np.array(trained)

array([[-1.81040299, -0.79967558,  0.58905655,  0.5216732 , -0.12743148],
       [-0.127406  , -1.4227078 ,  1.38958073, -0.10856824, -1.67363679],
       [ 1.74948514,  0.40018079,  1.26516676,  1.27793074,  0.90581268],
       [ 0.81082803,  0.59071714,  1.25839376,  1.30811751, -0.82108647],
       [ 0.09062722,  0.19807318, -0.91627866,  0.38640735,  0.02368835],
       [-0.31975693, -1.83046734,  0.29695246, -0.69859481,  0.67590463],
       [ 1.80997622, -2.14660215, -0.40159732, -0.04146788, -0.68597895],
       [ 1.47302425,  0.20840706, -0.95904928,  0.5442031 ,  1.98777115],
       [ 0.35210985, -1.07426488, -1.59163988, -1.28826213,  0.76360404],
       [ 0.67158389,  0.01300665, -0.88607883, -3.24122596,  0.0940977 ],
       [ 0.23196642, -0.6349678 ,  1.18822098, -0.03910354, -0.18538806],
       [ 0.96075815, -0.18448985,  0.69868511, -0.89909226, -0.78158206],
       [ 1.12219036, -1.7991401 , -1.33596385, -0.03429555, -0.37395012],
       [ 1.1546756 , -0.29695541, -1.0

In [148]:
np.array(non_trained)

array([[ 1.57651317, -0.24181476, -0.50905973,  0.80809194, -0.32008168],
       [ 0.62963688, -0.87307435, -1.15723062,  0.07763115,  0.77166176],
       [ 0.96675229,  1.15178859,  0.12776124, -0.05298454, -1.41787803],
       ...,
       [-0.26286599, -0.82421356,  0.25917768, -0.80786818,  1.51441991],
       [ 1.46801472, -1.27573347, -0.21419285, -2.75966334,  0.31739959],
       [-0.54773873,  0.55409747,  0.78499675, -0.68874353, -1.36398768]])

# 得出了结论，有相应的数据，对应的embedding才会被训练到

In [153]:
a = [190, 165,  89, 195,  36,  71,  59, 282, 149, 330, 241, 139, 248, 189,
        106,  21, 326, 303, 219, 223, 143, 283,  50, 265,  94,  44, 106, 323,
        233, 276, 191, 161, 178, 306, 255, 194,  77, 222,  65, 337, 209, 235,
         56, 289, 194,  61, 116, 284, 293, 208, 328, 328, 100, 279,   7, 196,
        215, 102,  88, 299, 191, 230, 234,  83, 275,  32, 318, 161,  80, 253,
        101, 270, 124,  52, 333, 200, 313,   9,  90,  26, 127,  66,  42, 270,
         31, 276,  39, 136, 251, 316, 184, 289,  62, 245,  43, 270, 331, 177,
        101, 315, 281, 219, 216,  42, 223,  76, 335, 325, 291, 198, 288, 106,
        293, 249,  50, 213, 312, 261, 318, 262, 289, 155,  89,  29, 207, 290,
        206, 158]
dic = {}
for i in a:
    if i not in dic:
        dic[i] = 1
    else:
        dic[i] += 1
dic

{190: 1,
 165: 1,
 89: 2,
 195: 1,
 36: 1,
 71: 1,
 59: 1,
 282: 1,
 149: 1,
 330: 1,
 241: 1,
 139: 1,
 248: 1,
 189: 1,
 106: 3,
 21: 1,
 326: 1,
 303: 1,
 219: 2,
 223: 2,
 143: 1,
 283: 1,
 50: 2,
 265: 1,
 94: 1,
 44: 1,
 323: 1,
 233: 1,
 276: 2,
 191: 2,
 161: 2,
 178: 1,
 306: 1,
 255: 1,
 194: 2,
 77: 1,
 222: 1,
 65: 1,
 337: 1,
 209: 1,
 235: 1,
 56: 1,
 289: 3,
 61: 1,
 116: 1,
 284: 1,
 293: 2,
 208: 1,
 328: 2,
 100: 1,
 279: 1,
 7: 1,
 196: 1,
 215: 1,
 102: 1,
 88: 1,
 299: 1,
 230: 1,
 234: 1,
 83: 1,
 275: 1,
 32: 1,
 318: 2,
 80: 1,
 253: 1,
 101: 2,
 270: 3,
 124: 1,
 52: 1,
 333: 1,
 200: 1,
 313: 1,
 9: 1,
 90: 1,
 26: 1,
 127: 1,
 66: 1,
 42: 2,
 31: 1,
 39: 1,
 136: 1,
 251: 1,
 316: 1,
 184: 1,
 62: 1,
 245: 1,
 43: 1,
 331: 1,
 177: 1,
 315: 1,
 281: 1,
 216: 1,
 76: 1,
 335: 1,
 325: 1,
 291: 1,
 198: 1,
 288: 1,
 249: 1,
 213: 1,
 312: 1,
 261: 1,
 262: 1,
 155: 1,
 29: 1,
 207: 1,
 290: 1,
 206: 1,
 158: 1}